### TRATAMENTO 1

Caminho:
- R:\CataliseInvestimentos\14. BigData Catalise\Staging\LiquidadosBaixados

Arquivo:
- 20250218_172853_TituloLiquidadosBaixados_BRISTOLFIDCMULTISSETORIAL.csv

Tratamentos:
1. Converter notação ciêntifica para número coluna (NUMERO CORRESPONDENTE)
2. Alterar para data abreviada coluna (DATA MOVIMENTO)

### TRATAMENTO 2

Caminho:
- R:\CataliseInvestimentos\14. BigData Catalise\Staging\LiquidadosBaixados

Arquivo:
- 20250218_181859_LiquidadosEBaixados_ÁGISFIDC_001.csv

Tratamento
1. Excluir linhas a direita


In [ ]:
import os
import time
import pandas as pd

from datetime import datetime, timedelta
datetime.today().strftime('Data: %Y-%m-%d Hora: %H:%m')

# DAYCOVAL

In [ ]:
arquivos = os.listdir("//172.31.8.209/Files/CataliseInvestimentos/14. BigData Catalise/Staging/LiquidadosBaixados")

csv_agis = list(filter(lambda x: any(item in x for item in ['ÁGISFIDC','CREDIAL','AGISFIDCNAOPADRONIZADO-TRANSFERIDO','AGISFIDCNAOPADRONIZADO']), arquivos)) # 

stop = 0
frames = []

while stop < len(csv_agis):
    file_path = f'//172.31.8.209/Files/CataliseInvestimentos/14. BigData Catalise/Staging/LiquidadosBaixados/{csv_agis[stop]}'
    df_csv_agis = pd.read_csv(file_path, delimiter=';', encoding='latin-1', dtype='str', index_col=False)
    frames.append(df_csv_agis)
    stop += 1

df_combined_agis = pd.concat(frames, ignore_index=True)

In [ ]:
df_combined_agis.FUNDO.drop_duplicates()

In [ ]:
# df_csv_agis = df_combined_agis[['FUNDO','CEDENTE','CPF/CNPJ','SACADO','CPF/CNPJ.1','SEU NUMERO','DOCUMENTO','TIPO RECEBIVEL',\
#                   'VALOR DE AQUISICÃO','VALOR DE PAGO','VALOR DE VENCIMENTO','AJUSTE','SITUACÃO DO RECEBÃVEL','DATA MOVIMENTO','DATA DA AQUISIÃÃO','DATA DE VENCIMENTO',\
#                   'TAXA','OCORRÃNCIA']] ## quando for historico

df_csv_agis = df_combined_agis[['FUNDO','CEDENTE','IDENTIFICACAO_CEDENTE','SACADO','IDENTIFICACAO_SACADO','SEU_NUMERO','DOCUMENTO','TIPO_RECEBIVEL',\
                  'VL_AQUISICAO','VALOR_PAGO','VALOR_VENCIMENTO','AJUSTE','ST_RECEBIVEL','DATA_MOVIMENTO','DATA_AQUISICAO','DATA_VENCIMENTO',\
                  'TX_AQUISICAO','TIPO_MOVIMENTO']]

## renomeia colunas
df_csv_agis.columns = ["NmFundo","NmCedente","DocCedente","NmSacado","DocSacado","Operacao","NuDocumento","TpRecebivel",
                       "VlrAquisicao","VlrPago","VlrVencimento","VlrAjuste","SituacaoRecebivel","DtMovimento","DtAquisicao",
                       "DtVencimento","TaxaAquisicao","Ocorrencia"]

date_columns = ['DtMovimento', 'DtVencimento',"DtAquisicao"]
df_csv_agis[date_columns] = df_csv_agis[date_columns].apply(pd.to_datetime, format="%d/%m/%Y")

## Tratando dados numéricos

columns_to_convert = ['VlrAquisicao', 'VlrVencimento','VlrPago', 'VlrAjuste','TaxaAquisicao']

def transformar_valor(valor):
    if pd.isna(valor):
        return valor
    valor_str = str(valor)
    valor_str = valor_str.replace('.', '').replace(',', '.')
    return float(valor_str)

for column in columns_to_convert:
    df_csv_agis[column] = df_csv_agis[column].apply(transformar_valor)
    
df_csv_agis.loc[df_csv_agis['NmFundo'] == 'ÁGIS FIDC', 'NmFundo'] = 'FIDC AGIS'
df_csv_agis.loc[df_csv_agis['NmFundo'] == 'AGIS FIDC NAO PADRONIZADO - TRANSFERIDO', 'NmFundo'] = 'FIDC AGIS'
df_csv_agis.loc[df_csv_agis['NmFundo'] == 'CREDIAL BANK PAN FIDC', 'NmFundo'] = 'FIDC CREDIAL'


# df_csv_agis[['NmFundo','VlrAquisicao', 'VlrPago','VlrVencimento', 'TaxaAquisicao']].head()
df_csv_agis[['NmFundo','VlrAquisicao', 'VlrVencimento','VlrPago', 'VlrAjuste','TaxaAquisicao']].head()


In [ ]:
df_csv_agis[['NmFundo','DtMovimento']].drop_duplicates()

In [ ]:
df_csv_agis['NmFundo'].count()

___________________

# AGROFORTE - APG

In [ ]:
arquivos = os.listdir("//172.31.8.209/Files/CataliseInvestimentos/14. BigData Catalise/Staging/LiquidadosBaixados")

csv_outros2 = list(filter(lambda x: any(item in x for item in ['AGROFORTEFIDC','APGFIDC', 'BRISTOLFIDCMULTISSETORIAL']), arquivos)) # 

stop = 0
frames = []

while stop < len(csv_outros2):
    file_path = f'//172.31.8.209/Files/CataliseInvestimentos/14. BigData Catalise/Staging/LiquidadosBaixados/{csv_outros2[stop]}'
    df_csv_outros2 = pd.read_csv(file_path, delimiter=';', encoding='latin-1', dtype='str', index_col=False)
    frames.append(df_csv_outros2)
    stop += 1

df_combined_outros2 = pd.concat(frames, ignore_index=True)
df_combined_outros2

In [ ]:
# df_csv_outros2 = df_combined_outros2[['FUNDO','CEDENTE', 'CPF/CNPJ', 'SACADO', 'CPF/CNPJ.1', 'SEU NUMERO', 'DOCUMENTO', 
#                                 'TIPO RECEBIVEL', 'VALOR DE AQUISICAO', 'VALOR DE PAGO', 'VALOR DE VENCIMENTO',  
#                                 'AJUSTE', 'SITUACAO DO RECEBIVEL', 'DATA MOVIMENTO',  'DATA DA AQUISICAO', 
#                                 'DATA DE VENCIMENTO', 'TAXA', 'OCORRENCIA']]
                                


## renomeia colunas
# df_csv_outros2.columns = ["NmFundo","NmCedente","DocCedente","NmSacado","DocSacado","Operacao","NuDocumento","TpRecebivel",
#                        "VlrAquisicao","VlrPago","VlrVencimento","VlrAjuste","SituacaoRecebivel","DtMovimento","DtAquisicao",
#                        "DtVencimento","TaxaAquisicao","Ocorrencia"]


df_csv_outros2 = df_combined_outros2[['FUNDO','CEDENTE','CPF/CNPJ','SACADO','CPF/CNPJ.1',
                                  'SEU NUMERO','DOCUMENTO','TIPO RECEBIVEL',
                                  'VALOR DE AQUISICAO','VALOR DE PAGO','VALOR DE VENCIMENTO','AJUSTE',
                                  'SITUACAO DO RECEBIVEL','DATA MOVIMENTO','DATA DA AQUISICAO','DATA DE VENCIMENTO',
                                  'TAXA','OCORRENCIA']]
                                

## renomeia colunas
df_csv_outros2.columns = ["NmFundo","NmCedente","DocCedente","NmSacado","DocSacado","Operacao","NuDocumento","TpRecebivel",
                       "VlrAquisicao","VlrPago","VlrVencimento","VlrAjuste","SituacaoRecebivel","DtMovimento","DtAquisicao",
                       "DtVencimento","TaxaAquisicao","Ocorrencia"]


date_columns = ['DtMovimento', 'DtVencimento',"DtAquisicao"]
df_csv_outros2[date_columns] = df_csv_outros2[date_columns].apply(pd.to_datetime, format="%d/%m/%Y")

## Tratando dados numéricos

columns_to_convert = ['VlrAquisicao', 'VlrVencimento','VlrPago', 'VlrAjuste','TaxaAquisicao']

def transformar_valor(valor):
    if pd.isna(valor):
        return valor
    valor_str = str(valor)
    valor_str = valor_str.replace('.', '').replace(',', '.')
    return float(valor_str)

for column in columns_to_convert:
    df_csv_outros2[column] = df_csv_outros2[column].apply(transformar_valor)

df_csv_outros2.loc[df_csv_outros2['NmFundo'] == 'AGROFORTE FIDC', 'NmFundo'] = 'FIDC AGROFORTE'
df_csv_outros2.loc[df_csv_outros2['NmFundo'] == 'APG FIDC', 'NmFundo'] = 'FIDC APG'
df_csv_outros2.loc[df_csv_outros2['NmFundo'] == 'BRISTOL FIDC MULTISSETORIAL', 'NmFundo'] = 'FIDC BRISTOL'
df_csv_outros2['VlrPago'] = df_csv_outros2['VlrPago']/100
df_csv_outros2[['NmFundo','VlrAquisicao', 'VlrPago','VlrVencimento', 'VlrAjuste','TaxaAquisicao']].head()

In [ ]:
df_csv_outros2.query("NmFundo == 'FIDC AGROFORTE'")[['NmFundo','DtMovimento','VlrAquisicao', 'VlrPago','VlrVencimento', 'VlrAjuste','TaxaAquisicao']].head()

In [ ]:
df_csv_outros2.query("NmFundo == 'FIDC BRISTOL'")[['NmFundo','DtMovimento','VlrAquisicao', 'VlrPago','VlrVencimento', 'VlrAjuste','TaxaAquisicao']].head()

In [ ]:
df_csv_outros2[['NmFundo','DtMovimento', 'TpRecebivel', 'Ocorrencia']].drop_duplicates().head()

___________________

# Master

In [ ]:
df_LiquidadosBaixados = pd.DataFrame()
LiquidadosBaixados = list(filter(lambda x: not any(item in x for item in ['SDL-FUNDODEINVESTIMENTOEMDIREITOSCREDITORIOS','ÁGIS2FUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOS',\
                                                                          'ÁGIS', 'AGROFORTEFIDC', 'APGFIDC','K-FINANCEFIDCNP', 'BLUEROCKETFIDC',\
                                                                         'GLOBALFUTURAFIDCRLFIDC','BELL','BRISTOLFIDCMULTISSETORIAL','TORONTOFUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOSDERESPONSABILIDADELIMITADA','VITTRAFUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOSDERESPONSABILIDADELIMITADANIXAUDITORESINDEPENDENTES','3RD-FUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOS','SMTAGROFUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOSDERESPONSABILIDADELIMITADA','MALBECFUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOS','IPE','BASAFIDCDERESPONSABILIDADELIMITADA',\
                                                                         'ALBAREDOFIDC','VERGINIA','ANVERESFUNDODEINVESTIMENTOEMDIREITOSCREDITORIOSDERESPONSABILIDADELIMITADA','USECORPCATÁLISEFUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOSSEGMENTOCOMERCIALDERESPLIMITADA', 'AF6','CREDIAL','ZINVESTFIDC','VISHNU',\
                                                                         'PrimeAgroFIDCRL','KERDOSFIDC','TERTONFUNDODEINVESTIMENTODIREITOSCREDITORIOSDERESPONSABILIDADELIMITADA','NR11FUNDODEINVESTIMENTOEMDIREITOSCREDITORIOSDERESPONSABILIDADELIMITADA','RHBCREDFUNDODEINVESTIMENTOEMDIREITOSCREDITORIOSDERESPONSABILIDADELIMITADA','SCFIDCRESPONSABILIDADEILIMITADA',\
                                                                         'DBANKFUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOSDERESPONSABILIDADELIMITADA',\
                                                                         'VELSO','FUTUROCAPITAL','AGROCETEFIDCNP','PAYCARGOFIDCNP',\
                                                                          'GREENWOODFUNDODEINVESTIMENTOEMDIREITOSCREDITORIOSNAOPADRONIZADOS',\
                                                                         'NINE','BONTEMPO','PINPAG','MASTRENN','CREDILOGII-FIDC','F2BANKFUNDODEINVESTIMENTOEMDIREITOSCREDITORIOS',\
                                                                         'MFGROUPFUNDODEINVESTIMENTOEMDIREITOSCREDITORIOS','CAPITALIZAFUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOSDERESPONSABILIDADELIMITADA',\
                                                                         'UKFFUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOS','BEFICFIDCDERESPONSABILIDADELIMITADA','ANILFUNDODEINVESTIMENTOEMDIREITOSCREDITORIOSDERESPONSABILIDADELIMITADA',\
                                                                         'LIBRACONSIGNADOFIDCDERL', 'LEXCAPITALFUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOSDERESPONSABILIDADELIMITADA', 'CONDOBEMFUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOSDERESPONSABILIDADELIMITADA']), arquivos))

stop = 0
frames = []

while stop < len(LiquidadosBaixados):
    file_path = f'//172.31.8.209/Files/CataliseInvestimentos/14. BigData Catalise/Staging/LiquidadosBaixados/{LiquidadosBaixados[stop]}'
    df_csv_liq = pd.read_csv(file_path, delimiter=';', encoding='latin-1', dtype='str', index_col=False)
    frames.append(df_csv_liq)
    stop += 1

df_combined_liqu = pd.concat(frames, ignore_index=True)



# df_LiquidadosBaixados = df_combined_liqu[['FUNDO','CEDENTE', 'CPF/CNPJ', 'SACADO', 'CPF/CNPJ.1', 'SEU NUMERO', 'DOCUMENTO', 'TIPO RECEBIVEL', 
# 'VALOR DE AQUISICAO', 'VALOR DE PAGO', 'VALOR DE VENCIMENTO',  'AJUSTE', 'SITUACAO DO RECEBIVEL',
#  'DATA MOVIMENTO',  'DATA DA AQUISICAO', 'DATA DE VENCIMENTO', 'TAXA', 'OCORRENCIA',]]

df_LiquidadosBaixados = df_combined_liqu[['FUNDO','CEDENTE','IDENTIFICACAO_CEDENTE','SACADO','IDENTIFICACAO_SACADO',
  'SEU_NUMERO','DOCUMENTO','TIPO_RECEBIVEL',
  'VL_AQUISICAO','VALOR_PAGO','VALOR_VENCIMENTO','AJUSTE',
  'ST_RECEBIVEL','DATA_MOVIMENTO','DATA_AQUISICAO','DATA_VENCIMENTO',
  'TX_AQUISICAO','TIPO_MOVIMENTO']]

# renomeia colunas
df_LiquidadosBaixados.columns = ["NmFundo","NmCedente","DocCedente","NmSacado","DocSacado","Operacao","NuDocumento","TpRecebivel",\
                                 "VlrAquisicao","VlrPago","VlrVencimento","VlrAjuste","SituacaoRecebivel","DtMovimento","DtAquisicao","DtVencimento","TaxaAquisicao","Ocorrencia"]
# Alterando o tipo das colunas de data

date_columns = ['DtMovimento', 'DtVencimento',"DtAquisicao"]
df_LiquidadosBaixados[date_columns] = df_LiquidadosBaixados[date_columns].apply(pd.to_datetime, format="%d/%m/%Y")

## Tratando dados numéricos

columns_to_convert = ['VlrAquisicao','VlrPago','VlrVencimento','VlrAjuste','TaxaAquisicao']

def transformar_valor(valor):
    if pd.isna(valor):
        return valor
    valor_str = str(valor)
    valor_str = valor_str.replace('.', '').replace(',', '.')
    return float(valor_str)

for column in columns_to_convert:
    df_LiquidadosBaixados[column] = df_LiquidadosBaixados[column].apply(transformar_valor)

df_LiquidadosBaixados.loc[df_LiquidadosBaixados['NmFundo'] == 'AGIS FIDC NAO PADRONIZADO - TRANSFERIDO', 'NmFundo'] = 'FIDC AGIS'


print(df_LiquidadosBaixados[["NmFundo","VlrAquisicao","VlrPago","VlrVencimento","VlrAjuste","TaxaAquisicao"]].head().to_string())

In [ ]:
df_LiquidadosBaixados[['NmFundo',"DtMovimento"]].drop_duplicates().sort_values("DtMovimento")

In [ ]:
df_LiquidadosBaixados.loc[
    df_LiquidadosBaixados['NmFundo'].str.contains('YELLOW CAPITAL FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA', case=False, na=False), 
    'NmFundo'
] = 'FIDC YELLOW'

In [ ]:
df_LiquidadosBaixados.loc[
    df_LiquidadosBaixados['NmFundo'].str.contains('BR8 FIDC', case=False, na=False), 
    'NmFundo'
] = 'FIDC BR8'



# Daycoval

In [ ]:
arquivos = os.listdir("//172.31.8.209/Files/CataliseInvestimentos/14. BigData Catalise/Staging/LiquidadosBaixados")

csv_outros = list(filter(lambda x: any(item in x for item in ['SDL-FUNDODEINVESTIMENTOEMDIREITOSCREDITORIOS','ÁGIS2FUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOS',\
                                                              'IPE', 'K-FINANCEFIDCNP', 'BLUEROCKETFIDC','GLOBALFUTURAFIDCRLFIDC',\
                                                              'BELLFUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOS','NR11FUNDODEINVESTIMENTOEMDIREITOSCREDITORIOSDERESPONSABILIDADELIMITADA','ANVERESFUNDODEINVESTIMENTOEMDIREITOSCREDITORIOSDERESPONSABILIDADELIMITADA','SMTAGROFUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOSDERESPONSABILIDADELIMITADA','BASAFIDCDERESPONSABILIDADELIMITADA',\
                                                              'ALBAREDOFIDC','VERGINIA','AF6','TORONTOFUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOSDERESPONSABILIDADELIMITADA','VITTRAFUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOSDERESPONSABILIDADELIMITADANIXAUDITORESINDEPENDENTES','3RD-FUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOS','RHBCREDFUNDODEINVESTIMENTOEMDIREITOSCREDITORIOSDERESPONSABILIDADELIMITADA','ZINVESTFIDC','VISHNUFIDC',\
                                                              'PrimeAgroFIDCRL','KERDOSFIDC','TERTONFUNDODEINVESTIMENTODIREITOSCREDITORIOSDERESPONSABILIDADELIMITADA','MALBECFUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOS','SCFIDCRESPONSABILIDADEILIMITADA',\
                                                             'DBANKFUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOSDERESPONSABILIDADELIMITADA',\
                                                              'VELSO','FUTUROCAPITAL','AGROCETEFIDCNP','USECORPCATÁLISEFUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOSSEGMENTOCOMERCIALDERESPLIMITADA','PAYCARGOFIDCNP',\
                                                             'GREENWOODFUNDODEINVESTIMENTOEMDIREITOSCREDITORIOSNAOPADRONIZADOS',\
                                                             'NINE','BONTEMPO','PINPAG','MASTRENN','CREDILOGII-FIDC','CAPITALIZA','F2BANKFUNDODEINVESTIMENTOEMDIREITOSCREDITORIOS',\
                                                             'MFGROUPFUNDODEINVESTIMENTOEMDIREITOSCREDITORIOS','UKFFUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOS',\
                                                              'BEFICFIDCDERESPONSABILIDADELIMITADA','ANILFUNDODEINVESTIMENTOEMDIREITOSCREDITORIOSDERESPONSABILIDADELIMITADA',\
                                                             'LIBRACONSIGNADOFIDCDERL', 'LEXCAPITALFUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOSDERESPONSABILIDADELIMITADA', 'CONDOBEMFUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOSDERESPONSABILIDADELIMITADA']), arquivos)) 

stop = 0
frames = []

while stop < len(csv_outros):
    file_path = f'//172.31.8.209/Files/CataliseInvestimentos/14. BigData Catalise/Staging/LiquidadosBaixados/{csv_outros[stop]}'
    df_csv_outros = pd.read_csv(file_path, delimiter=';', encoding='latin-1', dtype='str', index_col=False)
    frames.append(df_csv_outros)
    stop += 1

df_combined_outros = pd.concat(frames, ignore_index=True)


df_csv_outros = df_combined_outros[['FUNDO','CEDENTE','IDENTIFICACAO_CEDENTE','SACADO','IDENTIFICACAO_SACADO',
                                  'SEU_NUMERO','DOCUMENTO','TIPO_RECEBIVEL',
                                  'VL_AQUISICAO','VALOR_PAGO','VALOR_VENCIMENTO','AJUSTE',
                                  'ST_RECEBIVEL','DATA_MOVIMENTO','DATA_AQUISICAO','DATA_VENCIMENTO',
                                  'TX_AQUISICAO','TIPO_MOVIMENTO']]
                                


## renomeia colunas
df_csv_outros.columns = ["NmFundo","NmCedente","DocCedente","NmSacado","DocSacado","Operacao","NuDocumento","TpRecebivel",
                       "VlrAquisicao","VlrPago","VlrVencimento","VlrAjuste","SituacaoRecebivel","DtMovimento","DtAquisicao",
                       "DtVencimento","TaxaAquisicao","Ocorrencia"]

date_columns = ['DtMovimento', 'DtVencimento',"DtAquisicao"]
df_csv_outros[date_columns] = df_csv_outros[date_columns].apply(pd.to_datetime, format="%d/%m/%Y")

## Tratando dados numéricos

columns_to_convert = ['VlrAquisicao', 'VlrVencimento','VlrPago', 'VlrAjuste','TaxaAquisicao']

def transformar_valor(valor):
    if pd.isna(valor):
        return valor
    valor_str = str(valor)
    valor_str = valor_str.replace('.', '').replace(',', '.')
    return float(valor_str)

for column in columns_to_convert:
    df_csv_outros[column] = df_csv_outros[column].apply(transformar_valor)
    
    
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'SDL - FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS', 'NmFundo'] = 'FIDC SDL'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'ÁGIS 2 FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS', 'NmFundo'] = 'FIDC USECORP CATALISE'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'IPE FIDC NP', 'NmFundo'] = 'FIDC IPE'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'K-FINANCE FIDC NP', 'NmFundo'] = 'FIDC K-FINANCE'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'BLUE ROCKET FIDC', 'NmFundo'] = 'FIDC BLUE ROCKET'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'GLOBAL FUTURA FIDC RL FIDC', 'NmFundo'] = 'FIDC GLOBAL FUTURA'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'BELL FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS', 'NmFundo'] = 'FIDC BELL'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'FIDC IPE NP', 'NmFundo'] = 'FIDC IPE'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'BASA FIDC DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC BASA'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'ALBAREDO FIDC', 'NmFundo'] = 'FIDC ALBAREDO'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'AF6 FIDC NP', 'NmFundo'] = 'FIDC AF6'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'VERGINIA FIDC', 'NmFundo'] = 'FIDC VERGINIA'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'Z INVEST FIDC', 'NmFundo'] = 'FIDC Z INVEST'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'VISHNU FIDC', 'NmFundo'] = 'FIDC VISHNU'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'DFC FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS NP', 'NmFundo'] = 'FIDC DFC'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'Prime Agro FIDC RL', 'NmFundo'] = 'FIDC PRIME AGRO'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'KERDOS FIDC', 'NmFundo'] = 'FIDC KERDOS'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'SC FIDC RESPONSABILIDADE ILIMITADA', 'NmFundo'] = 'FIDC SC'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'DBANK FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC DBANK'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'FUTURO CAPITAL FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC FUTURO CAPITAL'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'VELSO - FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS NAO-PADRONIZADOS', 'NmFundo'] = 'FIDC VELSO'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'AGROCETE FIDC NP', 'NmFundo'] = 'FIDC AGROCETE'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'PAY CARGO FIDC NP', 'NmFundo'] = 'FIDC PAY CARGO'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'GREENWOOD FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS NAO PADRONIZADOS', 'NmFundo'] = 'FIDC GREENWOOD'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'NINE CAPITAL FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS', 'NmFundo'] = 'FIDC NINE CAPITAL'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'BONTEMPO FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC BONTEMPO'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'PINPAG FIDC', 'NmFundo'] = 'FIDC PINPAG'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'MASTRENN FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS', 'NmFundo'] = 'FIDC MASTRENN'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'CREDILOG II - FIDC', 'NmFundo'] = 'FIDC CREDILOG II'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'CAPITALIZA FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC CAPITALIZA'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'NINE CAPITAL FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS', 'NmFundo'] = 'FIDC NINE CAPITAL'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'MF GROUP FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS', 'NmFundo'] = 'FIDC MF GROUP'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'F2 BANK FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS', 'NmFundo'] = 'FIDC F2 BANK'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'UKF FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS', 'NmFundo'] = 'FIDC UKF'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'BEFIC FIDC DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC BEFIC'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'ANIL FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC ANIL'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'LIBRA CONSIGNADO FIDC DE RL', 'NmFundo'] = 'FIDC LIBRA'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'USECORP CATÁLISE FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS SEGMENTO COMERCIAL DE RESP LIMITADA', 'NmFundo'] = 'FIDC USECORP CATALISE'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'MALBEC FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS', 'NmFundo'] = 'FIDC MALBEC'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'RHB CRED FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC RHB'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'SMT AGRO FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC SMT AGRO'
df_csv_outros.loc[df_csv_outros['NmFundo'] == '3RD - FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS', 'NmFundo'] = 'FIDC 3RD'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'VITTRA FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA  NIX AUDITORES INDEPENDENTES', 'NmFundo'] = 'FIDC VITTRA'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'ANVERES  FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC ANVERES'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'NR11 FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC NR11'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'TERTON FUNDO DE INVESTIMENTO DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC TERTON'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'TORONTO FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC TORONTO'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'LEX CAPITAL FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC LEX'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'CONDOBEM FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC CONDOBEM'



df_csv_outros[["NmFundo","VlrAquisicao","VlrPago","VlrVencimento","VlrAjuste","TaxaAquisicao"]].head()

In [ ]:
df_csv_outros[['DtMovimento','NmFundo']].drop_duplicates()

_______

In [ ]:
print(f"df_LiquidadosBaixados: {len(df_LiquidadosBaixados)}\ndf_csv_agis: {len(df_csv_agis)}\ndf_csv_outros2: {len(df_csv_outros2)} \ndf_csv_outros: {len(df_csv_outros)}")

In [ ]:
df_final = pd.concat([df_csv_agis, df_csv_outros, df_csv_outros2, df_LiquidadosBaixados], ignore_index=True)

# Filtrando os valores nulos na coluna 'Ocorrencia'
df_nulos_ocorrencia = df_final[df_final['Ocorrencia'].isnull()]
df_nulos_ocorrencia

# OPCIONAL

In [ ]:
# df_final = pd.concat([df_csv_agis], ignore_index=True)

# Filtrando os valores nulos na coluna 'Ocorrencia'
# df_nulos_ocorrencia = df_final[df_final['Ocorrencia'].isnull()]
# df_nulos_ocorrencia

In [ ]:
df_final[["NmFundo"]].drop_duplicates()

In [ ]:
df_final[["NmFundo","VlrAquisicao","VlrPago","VlrVencimento","VlrAjuste","TaxaAquisicao"]].head()

In [ ]:
df_final[['DtMovimento','NmFundo']].drop_duplicates().sort_values('DtMovimento')

In [ ]:
qtdFundo = abs(len(df_final.NmFundo.drop_duplicates()) - len(arquivos))

if qtdFundo == 0:
    print("FALTANTES 0\n")
else:
    print(f"FALTANTES {qtdFundo}\n")
print(f"{df_final.NmFundo.drop_duplicates().sort_values()}")

In [ ]:
df_final.head()

In [ ]:
import mysql.connector
import pandas as pd


def insert_dataframe_to_mysql(df, connection, table_name):
    cursor = connection.cursor()
    for i, row in df.iterrows():
        sql = f"INSERT INTO {table_name} ({', '.join(row.index)}) VALUES ({', '.join(['%s'] * len(row))})"
        cursor.execute(sql, tuple(row))
    connection.commit()
    cursor.close()

# Conectar ao banco de dados MySQL
connection = mysql.connector.connect(
    host='catalise-bi-dados-cluster.cluster-ciaao5zm9beh.sa-east-1.rds.amazonaws.com',
    database='DW_STAGING',
    user='admin',
    password='yb222KraY7PTN0jbH7P3'
)

if connection.is_connected():
    print('Conectado ao MySQL\n')

    # Dividir o DataFrame em pedaços menores para evitar problemas de memória
    tamanho_do_pedaço = 10000  # Ajuste conforme necessário
    total_registros = len(df_final)

    for i in range(0, total_registros, tamanho_do_pedaço):
        pedaco_df = df_final.iloc[i:i+tamanho_do_pedaço]
        insert_dataframe_to_mysql(pedaco_df, connection, 'Stg_LiquidadosBaixados')
        print(f"Registros inseridos: {i + len(pedaco_df)} / {total_registros}")

    print('Concluído')
else:
    print('Falha na conexão ao MySQL')



# Fechar a conexão com o banco de dados
connection.close()

In [ ]:
import os
dir = '//172.31.8.209/Files/CataliseInvestimentos/14. BigData Catalise/Staging/LiquidadosBaixados'
for f in os.listdir(dir):
    os.remove(os.path.join(dir, f))

____________

# 

# TRATAMENTO 3 

### Caminho:
- R:\CataliseInvestimentos\14. BigData Catalise\Staging\Operacoes

### Arquivo:
- 20250219_172612_AquisicoesConsolidado_BRISTOLFIDCMULTISSETORIAL.csv

### Tratamento:
1. Alterar de notação ciêntifica para número as colunas (CPF_CNPJ SACADO, CPF_CNPJ_CEDENTE)
2. Alterar coluna "ENTRADA" para data abreviada
3. Alterar formato das colunas "CPF_CNPJSACADO" e "CPF_CNPJ_CEDENTE" para formato CNPJ (00\.000\.000\/0000\-00)

# 

____________

# Operações

In [ ]:
import os
import pandas as pd
import time
import pyodbc
from sqlalchemy import create_engine

def transformar_valor(valor):
    if pd.isna(valor):
        return valor
    valor_str = str(valor)
    valor_str = valor_str.replace('.', '').replace(',', '.')
    return float(valor_str)

# Obter lista de arquivos
diretorio_operacoes = "//172.31.8.209/Files/CataliseInvestimentos/14. BigData Catalise/Staging/Operacoes"
arquivos_ope = os.listdir(diretorio_operacoes)

# Inicializar um DataFrame vazio
df_operacoes_file = pd.DataFrame()

# Iterar sobre os arquivos e anexar ao DataFrame
for arquivo in arquivos_ope:
    file_path = os.path.join(diretorio_operacoes, arquivo)
    
    # Adicionar um bloco try-except para lidar com possíveis erros de leitura
    try:
        df_temp = pd.read_csv(file_path, delimiter=';', encoding='latin-1', dtype='str', index_col=False)
        df_operacoes_file = pd.concat([df_operacoes_file, df_temp], ignore_index=True)
    except pd.errors.EmptyDataError:
        # Lidar com arquivos vazios, se necessário
        print(f"Arquivo vazio: {arquivo}")
    except pd.errors.ParserError as e:
        # Lidar com erros de leitura, se necessário
        print(f"Erro na leitura do arquivo {arquivo}: {e}")
        
# df_operacoes_file = df_operacoes_file.drop_duplicates()
        
## 1 MASTER
df_operacoes = df_operacoes_file.query("FUNDO not in ['AGIS FIDC NAO PADRONIZADO - TRANSFERIDO','ÁGIS 2 FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS',\
                                       'AGROFORTE FIDC','AGIS FIDC NAO PADRONIZADO','APG FIDC',\
                                       'SC FIDC RESPONSABILIDADE ILIMITADA', 'IPE FIDC NP', 'K-FINANCE FIDC NP',\
                                       'SDL - FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS','BLUE ROCKET FIDC',\
                                       'Z INVEST FIDC','GLOBAL FUTURA FIDC RL FIDC','BRISTOL FIDC MULTISSETORIAL','3RD - FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS','BASA FIDC DE RESPONSABILIDADE LIMITADA',\
                                       'FIDC UKF', 'BELL FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS','TORONTO FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA','RHB CRED FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', 'ALBAREDO FIDC',\
                                       'Prime Agro FIDC RL','ÁGIS FIDC','SMT AGRO FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA', 'USECORP CATÁLISE FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS SEGMENTO COMERCIAL DE RESP LIMITADA','VERGINIA FIDC','MALBEC FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS','CREDIAL BANK PAN FIDC',\
                                       'BONTEMPO FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA',\
                                       'DBANK FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA',\
                                       'PINPAG FIDC','KERDOS FIDC','YELLOW CAPITAL FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA','FUTURO CAPITAL FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA',\
                                       'VELSO - FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS NAO-PADRONIZADOS','PAY CARGO FIDC NP',\
                                       'CAPITALIZA FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA',\
                                       'GREENWOOD FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS NAO PADRONIZADOS','TERTON FUNDO DE INVESTIMENTO DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA','BEFIC FIDC DE RESPONSABILIDADE LIMITADA',\
                                       'NINE CAPITAL FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS','AF6 FIDC NP','LIBRA CONSIGNADO FIDC DE RL',\
                                       'MASTRENN FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS','CREDILOG II - FIDC',\
                                       'NINE CAPITAL FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS','UKF FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS',\
                                       'MF GROUP FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS','F2 BANK FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS',\
                                       'NR11 FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA',\
                                       'ANVERES  FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA',\
                                       'VITTRA FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA  NIX AUDITORES INDEPENDENTES',\
                                       'ANIL FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', 'LEX CAPITAL FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA', 'SAGEL BANK FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA']")



## 3 DAYCOVAL
df_operacoes_agis2 = df_operacoes_file.query("FUNDO in ['AGIS FIDC NAO PADRONIZADO - TRANSFERIDO','ÁGIS FIDC','ÁGIS 2 FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS','SC FIDC RESPONSABILIDADE ILIMITADA','K-FINANCE FIDC NP',\
'SDL - FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS','BLUE ROCKET FIDC','Z INVEST FIDC','GLOBAL FUTURA FIDC RL FIDC',\
'BASA FIDC DE RESPONSABILIDADE LIMITADA','FIDC UKF','YELLOW CAPITAL FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA','USECORP CATÁLISE FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS SEGMENTO COMERCIAL DE RESP LIMITADA','MALBEC FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS', 'IPE FIDC NP','3RD - FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS', 'BELL FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS', 'ALBAREDO FIDC',\
'VERGINIA FIDC','CREDIAL BANK PAN FIDC','BONTEMPO FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA',\
'DBANK FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA',\
'PINPAG FIDC','Prime Agro FIDC RL','KERDOS FIDC','TORONTO FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA','TERTON FUNDO DE INVESTIMENTO DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA','SMT AGRO FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA','FUTURO CAPITAL FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA',\
'VELSO - FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS NAO-PADRONIZADOS','PAY CARGO FIDC NP','AGROCETE FIDC NP','RHB CRED FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA','CAPITALIZA FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA',\
'GREENWOOD FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS NAO PADRONIZADOS','BEFIC FIDC DE RESPONSABILIDADE LIMITADA',\
'NINE CAPITAL FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS','AF6 FIDC NP','LIBRA CONSIGNADO FIDC DE RL','MASTRENN FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS',\
'CREDILOG II - FIDC', 'NINE CAPITAL FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS','UKF FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS',\
'MF GROUP FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS','F2 BANK FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS','NR11 FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA',\
'ANVERES  FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA','VITTRA FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA  NIX AUDITORES INDEPENDENTES',\
'ANIL FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', 'LEX CAPITAL FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA', 'EJM FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA', 'SAGEL BANK FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA']")

## ALTERA NOME
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'SC FIDC RESPONSABILIDADE ILIMITADA', 'FUNDO'] = 'FIDC SC'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'SDL - FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS', 'FUNDO'] = 'FIDC SDL'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'K-FINANCE FIDC NP', 'FUNDO'] = 'FIDC K-FINANCE'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'BLUE ROCKET FIDC', 'FUNDO'] = 'FIDC BLUE ROCKET'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'GLOBAL FUTURA FIDC RL FIDC', 'FUNDO'] = 'FIDC GLOBAL FUTURA'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'BASA FIDC DE RESPONSABILIDADE LIMITADA', 'FUNDO'] = 'FIDC BASA'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'IPE FIDC NP', 'FUNDO'] = 'FIDC IPE'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'BELL FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS', 'FUNDO'] = 'FIDC BELL'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'VERGINIA FIDC', 'FUNDO'] = 'FIDC VERGINIA'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'CREDIAL BANK PAN FIDC', 'FUNDO'] = 'FIDC CREDIAL'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'BONTEMPO FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', 'FUNDO'] = 'FIDC BONTEMPO'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'DBANK FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA', 'FUNDO'] = 'FIDC DBANK'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'PINPAG FIDC', 'FUNDO'] = 'FIDC PINPAG'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'AGIS FIDC NAO PADRONIZADO - TRANSFERIDO', 'FUNDO'] = 'FIDC AGIS'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'Prime Agro FIDC RL', 'FUNDO'] = 'FIDC PRIME AGRO'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'KERDOS FIDC', 'FUNDO'] = 'FIDC KERDOS'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'FUTURO CAPITAL FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', 'FUNDO'] = 'FIDC FUTURO CAPITAL'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'VELSO - FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS NAO-PADRONIZADOS', 'FUNDO'] = 'FIDC VELSO'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'PAY CARGO FIDC NP', 'FUNDO'] = 'FIDC PAY CARGO'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'AGROCETE FIDC NP', 'FUNDO'] = 'FIDC AGROCETE'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'CAPITALIZA FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA', 'FUNDO'] = 'FIDC CAPITALIZA'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'GREENWOOD FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS NAO PADRONIZADOS', 'FUNDO'] = 'FIDC GREENWOOD'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'BEFIC FIDC DE RESPONSABILIDADE LIMITADA', 'FUNDO'] = 'FIDC BEFIC'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'AF6 FIDC NP', 'FUNDO'] = 'FIDC AF6'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'LIBRA CONSIGNADO FIDC DE RL', 'FUNDO'] = 'FIDC LIBRA CONSIGNADO'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'MASTRENN FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS', 'FUNDO'] = 'FIDC MASTRENN'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'CREDILOG II - FIDC', 'FUNDO'] = 'FIDC CREDILOG II'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'NINE CAPITAL FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS', 'FUNDO'] = 'FIDC NINE CAPITAL'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'UKF FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS', 'FUNDO'] = 'FIDC UKF'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'MF GROUP FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS', 'FUNDO'] = 'FIDC MF GROUP'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'F2 BANK FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS', 'FUNDO'] = 'FIDC F2 BANK'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'NR11 FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', 'FUNDO'] = 'FIDC NR11'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'ÁGIS FIDC', 'FUNDO'] = 'FIDC AGIS'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'ALBAREDO FIDC', 'FUNDO'] = 'FIDC ALBAREDO'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'Z INVEST FIDC', 'FUNDO'] = 'FIDC Z INVEST'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'ÁGIS 2 FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS', 'FUNDO'] = 'FIDC USECORP CATALISE'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'ANVERES  FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', 'FUNDO'] = 'FIDC ANVERES'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'VITTRA FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA  NIX AUDITORES INDEPENDENTES', 'FUNDO'] = 'FIDC VITTRA'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'ANIL FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', 'FUNDO'] = 'FIDC ANIL'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == '3RD - FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS', 'FUNDO'] = 'FIDC 3RD'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'MALBEC FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS', 'FUNDO'] = 'FIDC MALBEC'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'USECORP CATÁLISE FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS SEGMENTO COMERCIAL DE RESP LIMITADA', 'FUNDO'] = 'FIDC USECORP CATALISE'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'RHB CRED FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', 'FUNDO'] = 'FIDC RHB'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'SMT AGRO FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA', 'FUNDO'] = 'FIDC SMT AGRO'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'TERTON FUNDO DE INVESTIMENTO DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', 'FUNDO'] = 'FIDC TERTON'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'TORONTO FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA', 'FUNDO'] = 'FIDC TORONTO'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'YELLOW CAPITAL FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA', 'FUNDO'] = 'FIDC YELLOW'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'LEX CAPITAL FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA', 'FUNDO'] = 'FIDC LEX'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'BRISTOL FIDC MULTISSETORIAL', 'FUNDO'] = 'FIDC BRISTOL'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'EJM FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA', 'FUNDO'] = 'FIDC EJM'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'SAGEL BANK FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', 'FUNDO'] = 'FIDC SAGEL BANK'



## ALTERA NOME
## 2
df_operacoes_outros = df_operacoes_file.query("FUNDO in ['AGROFORTE FIDC','AGIS FIDC NAO PADRONIZADO','APG FIDC', 'BRISTOL FIDC MULTISSETORIAL']")

## ALTERA NOME
df_operacoes_outros.loc[df_operacoes_outros['FUNDO'] == 'AGIS FIDC NAO PADRONIZADO', 'FUNDO'] = 'FIDC AGIS'
df_operacoes_outros.loc[df_operacoes_outros['FUNDO'] == 'ÁGIS FIDC', 'FUNDO'] = 'FIDC AGIS'

df_operacoes_outros.loc[df_operacoes_outros['FUNDO'] == 'AGROFORTEFIDC', 'FUNDO'] = 'FIDC AGROFORTE'
df_operacoes_outros.loc[df_operacoes_outros['FUNDO'] == 'APG FIDC', 'FUNDO'] = 'FIDC APG'
df_operacoes_outros.loc[df_operacoes_outros['FUNDO'] == 'BRISTOL FIDC MULTISSETORIAL', 'FUNDO'] = 'FIDC BRISTOL'
df_operacoes_outros.loc[df_operacoes_outros['FUNDO'] == 'ZAB LEGACY FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', 'FUNDO'] = 'FIDC ZAB LEGACY'

print(f" df_operacoes: {len(df_operacoes)}\n df_operacoes_outros: {len(df_operacoes_outros)}\n df_operacoes_agis2: {len(df_operacoes_agis2)}")

df1 = df_operacoes[['FUNDO']].drop_duplicates()
df2 = df_operacoes_agis2[['FUNDO']].drop_duplicates()
df3 = df_operacoes_outros[['FUNDO']].drop_duplicates()

fundos_concatenados = pd.concat([df1, df2, df3]).drop_duplicates()


df = len(fundos_concatenados)

qtdFundo = abs(df - len(arquivos_ope))

if qtdFundo == 0:
    print("FALTANTES 0\n")      
else:
    print(f"FALTANTES {qtdFundo}\n")

In [ ]:
#df_operacoes[['FUNDO', 'ENTRADA']].drop_duplicates()
#df_operacoes_agis2[['ENTRADA','FUNDO']].drop_duplicates()
df_operacoes_outros[['ENTRADA','FUNDO']].drop_duplicates()

In [ ]:
df_operacoes = df_operacoes[['FUNDO',
                            'ENTRADA',
                            'CPF_CNPJ_CEDENTE',
                            'CPF_CNPJ SACADO' ,
                            'SEU NUMERO' ,
                            'NUMERO DOCUMENTO' ,
                            'TIPO RECEBIVEL' ,
                            'VALOR DE VENCIMENTO',
                            'VALOR DE COMPRA' ,
                            'DATA VENCIMENTO']]

# df_operacoes.columns = ['NmFundo','DtReferencia','DocCedente','DocSacado','Operacao','NuDocumento','TpRecebivel','VlrNominal','VlrAquisicao','VlrPago','VlrPresente','VlrAjuste','DtVencimento','DtAquisicao']
df_operacoes.columns = ['NmFundo','DtReferencia','DocCedente','DocSacado','Operacao','NuDocumento','TpRecebivel','VlrNominal','VlrAquisicao','DtVencimento']

date_columns = ['DtReferencia', 'DtVencimento']
df_operacoes[date_columns] = df_operacoes[date_columns].apply(pd.to_datetime, format="%d/%m/%Y", errors='coerce')

## Tratando dados numéricos

columns_to_convert = ['VlrNominal', 'VlrAquisicao']

for column in columns_to_convert:
    df_operacoes[column] = df_operacoes[column].apply(transformar_valor)

df_operacoes = df_operacoes.sort_values(by='DtReferencia',ascending=True)


In [ ]:
df_operacoes[['VlrNominal', 'VlrAquisicao']].head()

In [ ]:
df_operacoes_outros.head()

In [ ]:
print(df_operacoes[['NmFundo','DtReferencia']].drop_duplicates().head(300).to_string())

In [ ]:
df_operacoes.loc[
    df_operacoes['NmFundo'].str.contains('ZAB LEGACY FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', case=False, na=False), 
    'NmFundo'
] = 'FIDC ZAB LEGACY'

In [ ]:
df_operacoes.loc[
    df_operacoes['NmFundo'].str.contains('BR8 FIDC', case=False, na=False), 
    'NmFundo'
] = 'FIDC BR8'

In [ ]:
df_operacoes.loc[
    df_operacoes['NmFundo'].str.contains('VISHNU FIDC', case=False, na=False), 
    'NmFundo'
] = 'FIDC VISHNU'

In [ ]:
df_operacoes.info()

Mais rápido

In [ ]:
import mysql.connector
import pandas as pd
import time
start_time = time.time()

df_final = df_operacoes
 
# Função para inserir um pedaço do DataFrame na tabela MySQL em lote
def insert_dataframe_to_mysql_batch(df, connection, table_name, batch_size=100000):
    cursor = connection.cursor()
    cols = ",".join([str(i) for i in df.columns.tolist()])
    
    # Dividir o DataFrame em lotes
    for i in range(0, len(df), batch_size):
        batch_df = df.iloc[i:i+batch_size]
        values = [tuple(x) for x in batch_df.to_numpy()]
        placeholders = ",".join(["%s"] * len(df.columns))
        sql = f"INSERT INTO {table_name} ({cols}) VALUES ({placeholders})"
        cursor.executemany(sql, values)
        connection.commit()
    cursor.close()

# Conectar ao banco de dados MySQL
connection = mysql.connector.connect(
    host='catalise-bi-dados-cluster.cluster-ciaao5zm9beh.sa-east-1.rds.amazonaws.com',
    database='DW_CORPORATIVO',
    user='admin',
    password='yb222KraY7PTN0jbH7P3'
)

if connection.is_connected():
    print('Conectado ao MySQL')

    # Inserir DataFrame em lotes
    df_operacoes.fillna(0, inplace=True)
    insert_dataframe_to_mysql_batch(df_operacoes, connection, 'Ft_Operacoes', batch_size=10000)
    print('Concluído')
else:
    print('Falha na conexão ao MySQL')

# Fechar a conexão com o banco de dados
connection.close()

end_time = time.time()
# Calcula o tempo decorrido
elapsed_time = end_time - start_time
print(f"Tempo decorrido: {elapsed_time} segundos")

In [ ]:
# Contar valores NaN em cada coluna
nan_por_coluna = df_operacoes.isna().sum()

# Exibir o resumo
print(nan_por_coluna)

In [ ]:
df_operacoes.fillna(0, inplace=True)

# outros

In [ ]:
import time

import pyodbc
from sqlalchemy import create_engine


df_operacoes_outros = df_operacoes_outros[['FUNDO',
                            'ENTRADA',
                            'CPF_CNPJ_CEDENTE',
                            'CPF_CNPJ SACADO' ,
                            'SEU NUMERO' ,
                            'NUMERO DOCUMENTO' ,
                            'TIPO RECEBIVEL' ,
                            'VALOR DE VENCIMENTO',
                            'VALOR DE COMPRA' ,
                            'DATA VENCIMENTO']]


df_operacoes_outros.columns = ['NmFundo','DtReferencia','DocCedente','DocSacado','Operacao','NuDocumento','TpRecebivel','VlrNominal','VlrAquisicao','DtVencimento']


# Função para converter datas com base no nome do fundo
def converter_datas(row):
    if row['NmFundo'] in ['FIDC AGROFORTE', 'FIDC APG']:
        try:
            row['DtReferencia'] = pd.to_datetime(row['DtReferencia'], format='%Y-%m-%d', errors='coerce')
            row['DtVencimento'] = pd.to_datetime(row['DtVencimento'], format='%Y-%m-%d', errors='coerce')
        except:
            row['DtReferencia'] = pd.NaT
            row['DtVencimento'] = pd.NaT
            
    elif row['NmFundo'] == 'FIDC BRISTOL':
        try:
            row['DtReferencia'] = pd.to_datetime(row['DtReferencia'], format='%d/%m/%Y', errors='coerce')
            row['DtVencimento'] = pd.to_datetime(row['DtVencimento'], format='%d/%m/%Y', errors='coerce')
        except:
            row['DtReferencia'] = pd.NaT
            row['DtVencimento'] = pd.NaT
            
    return row

# Aplicando a função de conversão ao DataFrame
df_operacoes_outros = df_operacoes_outros.apply(converter_datas, axis=1)

# Garantindo que todas as datas estejam no formato Timestamp
df_operacoes_outros['DtReferencia'] = pd.to_datetime(df_operacoes_outros['DtReferencia'], errors='coerce')
df_operacoes_outros['DtVencimento'] = pd.to_datetime(df_operacoes_outros['DtVencimento'], errors='coerce')

## Tratando dados numéricos
columns_to_convert = ['VlrNominal', 'VlrAquisicao']

for column in columns_to_convert:
    df_operacoes_outros[column] = df_operacoes_outros[column].apply(transformar_valor)
    
df_operacoes_outros['VlrNominal'] = df_operacoes_outros['VlrNominal']/100
df_operacoes_outros['VlrAquisicao'] = df_operacoes_outros['VlrAquisicao']/100

# Ajustando nomes do fundos
df_operacoes_outros = df_operacoes_outros.sort_values(by='DtReferencia',ascending=True)
df_operacoes_outros.loc[df_operacoes_outros['NmFundo'] == 'AGIS FIDC NAO PADRONIZADO', 'NmFundo'] = 'FIDC AGIS'
df_operacoes_outros.loc[df_operacoes_outros['NmFundo'] == 'AGROFORTE FIDC', 'NmFundo'] = 'FIDC AGROFORTE'
df_operacoes_outros.loc[df_operacoes_outros['NmFundo'] == 'APG FIDC', 'NmFundo'] = 'FIDC APG'
df_operacoes_outros.NmFundo.drop_duplicates()

In [ ]:
df_operacoes_outros[['NmFundo','DtReferencia','DtVencimento','VlrNominal','VlrAquisicao']]

In [ ]:
df_operacoes_outros

In [ ]:
# df_operacoes_outros = df_operacoes_outros.query("DtReferencia=='2024-07-08'")

In [ ]:
import mysql.connector
import pandas as pd
import time
start_time = time.time()

df_final = df_operacoes_outros
 
# Função para inserir um pedaço do DataFrame na tabela MySQL em lote
def insert_dataframe_to_mysql_batch(df, connection, table_name, batch_size=100000):
    cursor = connection.cursor()
    cols = ",".join([str(i) for i in df.columns.tolist()])
    
    # Dividir o DataFrame em lotes
    for i in range(0, len(df), batch_size):
        batch_df = df.iloc[i:i+batch_size]
        values = [tuple(x) for x in batch_df.to_numpy()]
        placeholders = ",".join(["%s"] * len(df.columns))
        sql = f"INSERT INTO {table_name} ({cols}) VALUES ({placeholders})"
        cursor.executemany(sql, values)
        connection.commit()
    cursor.close()

# Conectar ao banco de dados MySQL
connection = mysql.connector.connect(
    host='catalise-bi-dados-cluster.cluster-ciaao5zm9beh.sa-east-1.rds.amazonaws.com',
    database='DW_CORPORATIVO',
    user='admin',
    password='yb222KraY7PTN0jbH7P3'
)

if connection.is_connected():
    print('Conectado ao MySQL')

    # Inserir DataFrame em lotes
    insert_dataframe_to_mysql_batch(df_final, connection, 'Ft_Operacoes', batch_size=10000)
    print('Concluído')
else:
    print('Falha na conexão ao MySQL')

# Fechar a conexão com o banco de dados
connection.close()

end_time = time.time()
# Calcula o tempo decorrido
elapsed_time = end_time - start_time
print(f"Tempo decorrido: {elapsed_time} segundos")

_________

# Daycoval


In [ ]:
import time

import pyodbc
from sqlalchemy import create_engine


df_operacoes_outros = df_operacoes_agis2[['FUNDO',
                            'ENTRADA',
                            'CPF_CNPJ_CEDENTE',
                            'CPF_CNPJ SACADO' ,
                            'SEU NUMERO' ,
                            'NUMERO DOCUMENTO' ,
                            'TIPO RECEBIVEL' ,
                            'VALOR DE VENCIMENTO',
                            'VALOR DE COMPRA' ,
                            'DATA VENCIMENTO']]


df_operacoes_outros.columns = ['NmFundo','DtReferencia','DocCedente','DocSacado','Operacao','NuDocumento','TpRecebivel','VlrNominal','VlrAquisicao','DtVencimento']

date_columns = ['DtReferencia', 'DtVencimento']
df_operacoes_outros[date_columns] = df_operacoes_outros[date_columns].apply(pd.to_datetime, format="%d/%m/%Y", errors='coerce')

## Tratando dados numéricos

columns_to_convert = ['VlrNominal', 'VlrAquisicao']

for column in columns_to_convert:
    df_operacoes_outros[column] = df_operacoes_outros[column].apply(transformar_valor)

df_operacoes_outros = df_operacoes_outros.sort_values(by='DtReferencia',ascending=True)

df_operacoes_outros.loc[df_operacoes_outros['NmFundo'] == 'ÁGIS 2 FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS', 'NmFundo'] = 'FIDC USECORP CATALISE SR'
df_operacoes_outros.loc[df_operacoes_outros['NmFundo'] == 'ALBAREDO FIDC', 'NmFundo'] = 'FIDC ALBAREDO'
df_operacoes_outros.loc[df_operacoes_outros['NmFundo'] == 'ÁGIS FIDC', 'NmFundo'] = 'FIDC AGIS'
df_operacoes_outros.loc[df_operacoes_outros['NmFundo'] == 'AGIS FIDC NAO PADRONIZADO - TRANSFERIDO', 'NmFundo'] = 'FIDC AGIS'

# INVESTIGAR ALBAREDO

df_operacoes_outros.loc[df_operacoes_outros['NmFundo'] == 'SC FIDC RESPONSABILIDADE ILIMITADA', 'NmFundo'] = 'FIDC SC'
df_operacoes_outros.loc[df_operacoes_outros['NmFundo'] == 'IPE FIDC NP', 'NmFundo'] = 'FIDC IPE'

df_operacoes_outros.loc[df_operacoes_outros['NmFundo'] == 'Z INVEST FIDC', 'NmFundo'] = 'FIDC Z INVEST'
df_operacoes_outros.loc[df_operacoes_outros['NmFundo'] == 'BLUE ROCKET FIDC', 'NmFundo'] = 'FIDC BLUE ROCKET'
df_operacoes_outros.loc[df_operacoes_outros['NmFundo'] == 'FIDC BASA', 'NmFundo'] = 'FIDC BASA'


In [ ]:
df_operacoes_outros.NmFundo.drop_duplicates()

In [ ]:
df_operacoes_outros[['VlrNominal', 'VlrAquisicao','DocCedente', 'DocSacado']]

In [ ]:
df_operacoes_outros[['DtReferencia', 'NmFundo']].drop_duplicates()

In [ ]:
df_final

In [ ]:
# Contar valores NaN em cada coluna
nan_por_coluna = df_final.isna().sum()

# Exibir o resumo
print(nan_por_coluna)

In [ ]:
df_final.fillna(0, inplace=True)

In [ ]:
import mysql.connector
import pandas as pd
import time

start_time = time.time()

df_final = df_operacoes_outros

# Função para inserir um pedaço do DataFrame na tabela MySQL em lote
def insert_dataframe_to_mysql_batch(df, connection, table_name, batch_size=100000):
    # Preencher valores NaN com uma string vazia
    df = df.fillna('')

    cursor = connection.cursor()
    cols = ",".join([str(i) for i in df.columns.tolist()])
    
    # Dividir o DataFrame em lotes
    for i in range(0, len(df), batch_size):
        batch_df = df.iloc[i:i+batch_size]
        values = [tuple(x) for x in batch_df.to_numpy()]
        placeholders = ",".join(["%s"] * len(df.columns))
        sql = f"INSERT INTO {table_name} ({cols}) VALUES ({placeholders})"
        cursor.executemany(sql, values)
        connection.commit()
    cursor.close()

# Conectar ao banco de dados MySQL
connection = mysql.connector.connect(
    host='catalise-bi-dados-cluster.cluster-ciaao5zm9beh.sa-east-1.rds.amazonaws.com',
    database='DW_CORPORATIVO',
    user='admin',
    password='yb222KraY7PTN0jbH7P3'
)

if connection.is_connected():
    print('Conectado ao MySQL')

    # Inserir DataFrame em lotes
    insert_dataframe_to_mysql_batch(df_final, connection, 'Ft_Operacoes', batch_size=10000)
    print('Concluído')
else:
    print('Falha na conexão ao MySQL')

# Fechar a conexão com o banco de dados
connection.close()

end_time = time.time()
# Calcula o tempo decorrido
elapsed_time = end_time - start_time
print(f"Tempo decorrido: {elapsed_time} segundos")

In [ ]:
import os
dir = '//172.31.8.209/Files/CataliseInvestimentos/14. BigData Catalise/Staging/Operacoes'
for f in os.listdir(dir):
    os.remove(os.path.join(dir, f))